In [18]:
import pickle
import numpy as np
import pandas as pd
from fastapi import FastAPI
from pydantic import BaseModel

In [19]:
with open("best_model.pkl", "rb") as f:
    model = pickle.load(f)

In [20]:
app = FastAPI()

In [21]:
from typing import Literal
class InputData(BaseModel):
    Gender: Literal['Male', 'Female']
    Age: int
    Height: float
    Weight: float
    family_history_with_overweight: Literal['yes', 'no']
    FAVC: Literal['yes', 'no']
    FCVC: float
    NCP: float
    CAEC: Literal['Sometimes', 'Frequently', 'no', 'Always']
    SMOKE: Literal['yes', 'no']
    CH2O: float
    SCC: Literal['yes', 'no']
    FAF: float
    TUE: float
    CALC: Literal['no', 'Sometimes', 'Frequently', 'Always']
    MTRANS: Literal['Public_Transportation', 'Bike', 'Automobile', 'Motorbike', 'Walking']

In [22]:
def predict(data: InputData):
    input_df = pd.DataFrame([data.dict()])

    input_df = pd.get_dummies(input_df)

    with open("X_columns.pkl", "rb") as f:
        X_columns = pickle.load(f)

    input_df = input_df.reindex(columns=X_columns, fill_value=0)

    with open("scaler.pkl", "rb") as f:
        scaler = pickle.load(f)

    input_scaled = scaler.transform(input_df)

    prediction = model.predict(input_scaled)

    with open("label_encoder.pkl", "rb") as f:
        le = pickle.load(f)

    predicted_label = le.inverse_transform(prediction)[0]

    return {"prediction": predicted_label}

In [23]:
@app.post("/predict")
def predict_endpoint(data: InputData):
    return predict(data)